<a href="https://colab.research.google.com/github/sa1manhaidr/autogen/blob/main/SMS_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
!pip install XlsxWriter
!pip install openpyxl==3.0.7
!pip install xlrd==1.2.0
clear_output()
import xlsxwriter
from xlsxwriter.utility import xl_range
from xlsxwriter.utility import xl_rowcol_to_cell
import datetime
import pandas as pd 
import numpy as np
import os # 
import openpyxl # 
from openpyxl.styles import Border, Side # Alignment
from openpyxl.styles import numbers
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
import io
from itertools import cycle
from google.colab import files


##################################DATE#######################
Previous_Date = datetime.datetime.today() - datetime.timedelta(days=1)
Previous_Date_Formatted = Previous_Date.strftime ('%d %B %Y')
Previous_Date_TAB=Previous_Date.strftime ('%d - %B')
############################################################

print("Telebiz SMS Report Generator")

print("Upload Client/Vendor report in xlsx")

SMSREAD = files.upload()
for Cmin in SMSREAD.keys():
    name=Cmin

smsraw = pd.read_excel(io.BytesIO(SMSREAD[Cmin]))




new_header = smsraw.iloc[0]
new_header = smsraw.iloc[0]
smsraw = smsraw[1:]
smsraw.columns = new_header

smsraw=smsraw[smsraw["Client name"].str.contains("Total") == False]
smsraw=smsraw[smsraw["Country"].str.contains("monitoring") == False]

########SORTING BY COUNTRY############
smsCust=smsraw[["Country","Client name", "Client Product" , "Billable (C)"]]
smsCust= smsCust[smsCust['Billable (C)'] != 0]
smsCust=smsCust[smsCust["Country"].str.contains("Pakistan") == False]
Cusadd=smsCust.groupby(['Country','Client name','Client Product'])['Billable (C)'].sum().reset_index()
Cusadd=Cusadd.sort_values(by=['Country','Billable (C)'] , ascending=[True,False] ) [["Country","Client name", "Client Product" , "Billable (C)"]]
TotalCus1=Cusadd['Billable (C)'].sum()
######
smsProv=smsraw [["Country","Vendor name", "Vendor Product" , "Billable (V)"]]
smsProv= smsProv[smsProv['Billable (V)'] != 0]
smsProv=smsProv[smsProv["Country"].str.contains("Pakistan") == False]
Provadd=smsProv.groupby(['Country','Vendor name','Vendor Product'])['Billable (V)'].sum().reset_index()
Provadd=Provadd.sort_values(by=['Country','Billable (V)'] , ascending=[True,False] ) [["Country","Vendor name", "Vendor Product" , "Billable (V)"]]
TotalPro1=Provadd['Billable (V)'].sum()

########SORTING BY Destination############
smsDestCust=smsraw.sort_values(by=['Country','Billable (C)'] , ascending=[True,False] ) [["Country","Net","Client name", "Client Product" , "Billable (C)","Vendor name", "Vendor Product" , "Billable (V)"]]
smdpk=smsDestCust.loc[smsDestCust['Country'].str.contains('Pakistan')]
smdpk["Net"]= smdpk["Net"].replace(["Mobilink","Telenor Pakistan","Ufone","Warid Telecom","ZONG"], ["Pakistan Mobilink","Pakistan Telenor","Pakistan Ufone","Pakistan Warid","Pakistan Zong"])
smsDcus=smdpk.sort_values(by=['Net','Billable (C)'] , ascending=[True,False] ) [["Net","Client name", "Client Product" , "Billable (C)"]]
smsDcus= smsDcus[smsDcus['Billable (C)'] != 0]
DesCusadd=smsDcus.groupby(['Net','Client name','Client Product'])['Billable (C)'].sum().reset_index()
TotalCus2=smsDcus['Billable (C)'].sum()

smsDProv=smdpk.sort_values(by=['Net','Billable (C)'] , ascending=[True,False] )[["Net","Vendor name", "Vendor Product" , "Billable (V)"]]
smsDProv= smsDProv[smsDProv['Billable (V)'] != 0]
DesProvadd=smsDProv.groupby(['Net','Vendor name','Vendor Product'])['Billable (V)'].sum().reset_index()
TotalPro2=DesProvadd['Billable (V)'].sum()

#############COuntryListing#############
CL=Cusadd['Country'].tolist()
F_CL=dict.fromkeys(CL)
Country_list=list(F_CL)

#############DestinationListing#############
DL=smsDcus['Net'].tolist()
F_DL=dict.fromkeys(DL)
Destination_list=list(F_DL)

print(name)
###########################################
empt_rows2=[]
TT_rows2=[]
zc=0
rw_en2=0
appended_data = []
i = 0
try:
 while i < len(Country_list):
    d=Cusadd.loc[Cusadd['Country'].str.contains(Country_list[i])].sort_values(by='Billable (C)', ascending=False)
    e=Provadd.loc[Provadd['Country'].str.contains(Country_list[i])].sort_values(by='Billable (V)', ascending=False)
    eselect=e[['Vendor name', 'Vendor Product', 'Billable (V)']]
    d.reset_index(drop=True, inplace=True)
    eselect.reset_index(drop=True, inplace=True)
    w=pd.concat([d,eselect]  ,axis=1  ) 
    w.loc[w.shape[0]] = [None, None, None, None, None, None, None]
    w.loc[w.shape[0]] = [None, None, None, None, None, None, None]
    w.at['Total', 'Billable (C)'] = w['Billable (C)'].sum()
    w.at['Total', 'Client Product'] = 'Total'
    w.at['Total', 'Billable (V)'] = w['Billable (V)'].sum()
    w.at['Total', 'Vendor Product'] = 'Total'
    w['Country'] = w['Country'].fillna(Country_list[i])
    rw_en2=rw_en2+len(w)+1
    if (i != len(Country_list)-1):
         w.loc[w.shape[0]] = [None, None, None, None, None, None, None]
    empt_rows2.append(rw_en2)
    TT_rows2.append(zc)
    appended_data.append(w)
    i = i + 1

 appended_data = pd.concat(appended_data)

 appended_data.reset_index(drop=True ,inplace=True )

 appended_data.columns= ['Destination','Client name','Client Product','Billable (C)','Vendor name','Vendor Product','Billable (V)']

except ValueError:
    pass
################################################
empt_rows=[]
rw_en=0
appended_data2 = []
j = 0

while j < len(Destination_list):
   x=DesCusadd.loc[DesCusadd['Net'].str.contains(Destination_list[j])].sort_values(by='Billable (C)', ascending=False)
   y=DesProvadd.loc[DesProvadd['Net'].str.contains(Destination_list[j])].sort_values(by='Billable (V)', ascending=False)
   yselect=y[['Vendor name', 'Vendor Product', 'Billable (V)']]
   x.reset_index(drop=True, inplace=True)
   yselect.reset_index(drop=True, inplace=True)
   w=pd.concat([x,yselect]  ,axis=1  )
   w.loc[w.shape[0]] = [None, None, None, None, None, None, None]
   w.loc[w.shape[0]] = [None, None, None, None, None, None, None]
   w.at['Total', 'Billable (C)'] = w['Billable (C)'].sum()
   w.at['Total', 'Client Product'] = 'Total'
   w.at['Total', 'Billable (V)'] = w['Billable (V)'].sum()
   w.at['Total', 'Vendor Product'] = 'Total'
   w['Net'] = w['Net'].fillna(Destination_list[j])
   rw_en=rw_en+len(w)+1
   w.loc[w.shape[0]] = [None, None, None, None, None, None, None]
   empt_rows.append(rw_en)
   appended_data2.append(w)
   j = j + 1

appended_data2 = pd.concat(appended_data2)
appended_data2.reset_index(drop=True ,inplace=True )
appended_data2.columns = ['Destination','Client name','Client Product','Billable (C)','Vendor name','Vendor Product','Billable (V)']


cbsm=appended_data2.append(appended_data)
cbsm.reset_index(drop=True ,inplace=True )
cbsm.index += 5

e1=[x+5 for x in empt_rows ]
e2=[x+len(appended_data2)+5 for x in empt_rows2 ]

emprow=e1+e2
tt_rws = emprow.copy()
emprow.pop(-1)





###################################################
startCells = [6]
for row in range(7,len(cbsm)+6):
    if (cbsm.loc[row-1,'Destination'] != cbsm.loc[row-2,'Destination']):
        startCells.append(row)
        

lastRow = len(cbsm)+5
TT_row=lastRow+4

writer = pd.ExcelWriter('Daily SMS Termination Report.xlsx', engine='xlsxwriter')
cbsm.to_excel(writer, sheet_name=Previous_Date_TAB , startrow=6  , header=False ,  index=False)



#print(Destination_list)
#cbsm.to_csv('spr1.csv')


workbook = writer.book
worksheet = writer.sheets[Previous_Date_TAB]
merge_format = workbook.add_format({'bold': True,'align': 'center', 'valign': 'vcenter', 'border': 2})
merge_format2 = workbook.add_format({'bold': True, 'border': 2 , 'num_format': '#,##0'})
for row in startCells:
    try:
        endRow = startCells[startCells.index(row)+1]-1
        if row == endRow:
            worksheet.write(row, 0, cbsm.loc[row-1,'Destination'], merge_format)
        else:
            worksheet.merge_range(row, 0, endRow, 0, cbsm.loc[row-1,'Destination'], merge_format)
           
    except IndexError:
        if row == lastRow:
            worksheet.write(row, 0, cbsm.loc[row-1,'Destination'], merge_format)
            
        else:
            worksheet.merge_range(row, 0, lastRow, 0, cbsm.loc[row-1,'Destination'], merge_format)
            
            

for row in emprow:
    
    worksheet.merge_range(row, 0, row, 6,'',  merge_format)  


    
###############################################################################3    
numform = workbook.add_format({'num_format': '#,##0'})         
worksheet.set_column('B:B', 41)
worksheet.set_column('E:E', 41)
worksheet.set_column('C:G', 20)
worksheet.set_column('A:A', 28)
worksheet.set_column('D:D',16, numform)
worksheet.set_column('G:G',16, numform)

merge_format_heading=workbook.add_format({
    
    'border': 2,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#8EA9DB',
     'font_size': '16',
    'font_name':'Calibri'
})

merge_format_Subheading=workbook.add_format({
    
    'border': 2,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#8EA9DB',
     'font_size': '9',
     'font_name':'Tahoma'
})

merge_format_Subheading2=workbook.add_format({
    
    'border': 2,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#8EA9DB',
     'font_size': '9',
     'font_name':'Tahoma'
})









worksheet.merge_range('A1:G2','SMS Termination Report ' + str(Previous_Date_Formatted), merge_format_heading)
worksheet.merge_range ('A3:A6', 'Destination', merge_format_Subheading)
worksheet.merge_range ('B3:D4', 'Customer SMS', merge_format_Subheading)
worksheet.merge_range ('E3:G4', 'Supplier SMS', merge_format_Subheading)
worksheet.merge_range ('B5:B6', 'Name', merge_format_Subheading2)
worksheet.merge_range ('E5:E6', 'Name', merge_format_Subheading2)
worksheet.merge_range ('C5:C6', 'Service', merge_format_Subheading2)
worksheet.merge_range ('F5:F6', 'Service', merge_format_Subheading2)
worksheet.merge_range ('D5:D6', 'Total SMS', merge_format_Subheading2)
worksheet.merge_range ('G5:G6', 'Total SMS', merge_format_Subheading2)
#######Total Minutes
TotalCus= TotalCus1+TotalCus2
TotalProv= TotalPro1+TotalPro2



worksheet.merge_range(TT_row, 1, TT_row, 2, 'Total', merge_format)
worksheet.write(TT_row, 3,TotalCus, merge_format2)
worksheet.merge_range(TT_row, 4, TT_row, 5, 'Total', merge_format)
worksheet.write(TT_row, 6,TotalProv, merge_format2)
    
writer.save()
#writer.close()
writer.handles = None

wb = load_workbook('Daily SMS Termination Report.xlsx')
sheet = wb.active


borderThick = Border(left=Side(style='medium'), 
                    right=Side(style='medium'), 
                    top=Side(style='medium'), 
                    bottom=Side(style='medium')) #

borderThin = Border(left=Side(style='thin'),
                    right=Side(style='medium'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin')) # 

borderTop = Border(top=Side(style='thick'),
                    left=Side(style='thin'),
                    right=Side(style='thin'), 
                    bottom=Side(style='thin'))

borderBottom = Border(bottom=Side(style='medium'),
                    left=Side(style='thin'),
                    right=Side(style='medium'),
                    top=Side(style='thin'))

borderLeft = Border(left=Side(style='thick'),
                    right=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin')) # 
                    
borderRight = Border(right=Side(style='medium'),
                    left=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin')) # 

borderTopLeft = Border(left=Side(style='thick'),
                    right=Side(style='thin'),
                    top=Side(style='thick'),
                    bottom=Side(style='thin')) #

borderTopRight = Border(left=Side(style='thin'),
                    right=Side(style='medium'),
                    top=Side(style='medium'),
                    bottom=Side(style='thin')) #

borderBottomLeft = Border(left=Side(style='thick'),
                    right=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thick')) #

borderBottomRight = Border(left=Side(style='thin'),
                    right=Side(style='medium'),
                    top=Side(style='thin'),
                    bottom=Side(style='medium')) #

borderSide = Side(border_style='medium')  # applies to sides of each cell

startCells.pop(0) 

startCellsadd1 = [x+1 for x in startCells]



# Set thin borders on entire sheet

ERow=int(lastRow)+1


rows = range(7, ERow)
rowTop = startCells
rowBot = int(ERow)
colLeft = int(1)
colRight = int(7)
columns = range(1, 7)
columnsB=range(1,8)
for row in rows:
    for col in columns:
        sheet.cell(row, col).border = borderThin
   



#Set thick outer borders




for row in rows:
    for col in columns:
        sheet.cell(rowBot, col).border = borderBottom
        sheet.cell(row, colRight).border = borderRight
        sheet.cell(ERow,7).border = borderBottomRight
                
for row in startCells:
    for col in columnsB:
        sheet.cell(row, col).border = borderBottom
        

for row in tt_rws:
    for col in columnsB:
        sheet.cell(row, col).border = borderThick
        sheet.cell(row, col).font = Font(bold=True)
        
        
wb.save('Daily SMS Termination Report.xlsx')
print("Done!!")
clear_output()
files.download('Daily SMS Termination Report.xlsx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>